# Energy Calculators (Green Algorithms, ML CO2 Impact)
Parameters needed include the duration of the experiment and the average utilization.

## Imports and variables

In [1]:
REPOSITORY_DIR = "../../" # TO MODIFY
prefix = [REPOSITORY_DIR + "results/GPU_20_04/", REPOSITORY_DIR + "results/GPU_19_04/"] # ADD AS MANY FOLDER AS YOU NEED
#prefix = [REPOSITORY_DIR + "results/CPU_08_11/"]

import sys
sys.path.append(REPOSITORY_DIR + "code/")
import pandas as pd
import matplotlib.pyplot as plt
from utils.process_results import add_benchmark_id_to_merged_timeseries, cleaning_table

In [2]:
energy_scope_df = pd.concat([pd.read_csv(file + 'es_ts.csv') for file in prefix])
watt_df = pd.concat([pd.read_csv(file + 'g5k_metrics.csv') for file in prefix])
exp_table = pd.concat([pd.read_csv(file + 'processed_table.csv') for file in prefix])

exp_table = cleaning_table(exp_table, watt_df, energy_scope_df)

grouped_watt_df = watt_df.groupby(by=['timestamp_sec']).mean().reset_index()
merged_df = pd.merge(energy_scope_df,grouped_watt_df, on='timestamp_sec', how='outer').sort_values(by=['timestamp_sec'])

merged_df['wattmetre_es_diff'] = abs(merged_df['wattmetre_power_watt'] - merged_df['data.data.etotal(W)'])

b_df = add_benchmark_id_to_merged_timeseries(exp_table, merged_df)

## Compute duration

In [3]:
exp_table['benchmark_duration(diff)'] = exp_table['gpu_0_stop_benchmark'] - exp_table['gpu_0_start_benchmark']

In [5]:
exp_table[(exp_table['tool_on_one_process']==False)&(exp_table['tool_name']=='energy scope')].groupby(by=['gpu_0_appli','gpu_0_appli_class'])[['benchmark_duration(diff)']].mean()

,,benchmark_duration(diff)
gpu_0_appli,gpu_0_appli_class,
ep,E,68.786320
idle,sh,60.008526
lu,D,204.842728
mg,D,160.916385


In [6]:
exp_table[(exp_table['tool_on_one_process']==False)&(exp_table['tool_name']=='energy scope')].groupby(by=['gpu_0_appli','gpu_0_appli_class'])[['benchmark_duration(diff)']].std()

,,benchmark_duration(diff)
gpu_0_appli,gpu_0_appli_class,
ep,E,0.062094
idle,sh,0.000231
lu,D,0.058560
mg,D,6.025193


In [7]:
exp_table[(exp_table['tool_on_one_process']==False)&(exp_table['tool_name']=='energy scope')].groupby(by=['gpu_0_appli','gpu_0_appli_class'])[['wattmetre_power_watt_energy_consumption(kWh)']].mean()

,,wattmetre_power_watt_energy_consumption(kWh)
gpu_0_appli,gpu_0_appli_class,
ep,E,0.048938
idle,sh,0.014527
lu,D,0.112923
mg,D,0.047998


In [8]:
exp_table[(exp_table['tool_on_one_process']==False)&(exp_table['tool_name']=='energy scope')].groupby(by=['gpu_0_appli','gpu_0_appli_class'])[['bmc_node_power_watt_energy_consumption(kWh)']].mean()

,,bmc_node_power_watt_energy_consumption(kWh)
gpu_0_appli,gpu_0_appli_class,
ep,E,0.042895
idle,sh,0.012070
lu,D,0.101655
mg,D,0.042220


## Values from Green Algorithm and ML CO2 Impact

In [15]:
online_tools = {}
# GPU benchmarks
online_tools['Wattmetre'] = {}
online_tools['Wattmetre']['EP'] = 0.048796 * 1000 * 3600
online_tools['Wattmetre']['LU'] = 0.112751 * 1000 * 3600
online_tools['Wattmetre']['MG'] = 0.046730 * 1000 * 3600
online_tools['Wattmetre']['idle'] = 0.014501 * 1000 * 3600

online_tools['BMC'] = {}
online_tools['BMC']['EP'] = 0.042313 * 1000 * 3600
online_tools['BMC']['LU'] = 0.101703 * 1000 * 3600
online_tools['BMC']['MG'] = 0.040750 * 1000 * 3600
online_tools['BMC']['idle'] = 0.011970 * 1000 * 3600

# CPU benchmarks
online_tools['Wattmetre'] = {}
online_tools['Wattmetre']['EP'] = 0.016115 * 1000 * 3600
online_tools['Wattmetre']['LU'] = 0.009754 * 1000 * 3600
online_tools['Wattmetre']['MG'] = 0.042828 * 1000 * 3600
online_tools['Wattmetre']['idle'] = 0.019227 * 1000 * 3600

online_tools['BMC'] = {}
online_tools['BMC']['EP'] = 0.013340 * 1000 * 3600
online_tools['BMC']['LU'] = 0.007420 * 1000 * 3600
online_tools['BMC']['MG'] = 0.037230 * 1000 * 3600
online_tools['BMC']['idle'] = 0.015979 * 1000 * 3600

In [59]:
# GPU
online_tools['Green Algorithm'] = {}
online_tools['Green Algorithm']['EP'] = (43.18 * 68 / 60) * 3600 # converting to joules https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=1&memory=512&platformType=localServer
online_tools['Green Algorithm']['LU'] = (31.18 * 204 / 60) * 3600 # https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=0.7&memory=512&platformType=localServer
online_tools['Green Algorithm']['MG'] = (14.26 * 157 / 60) * 3600 # https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0.2&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=0.25&memory=512&platformType=localServer
online_tools['Green Algorithm']['idle'] = (2.29 * 157 / 60) * 3600 # https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=0&memory=512&platformType=localServer

# CPU
online_tools['Green Algorithm'] = {}
online_tools['Green Algorithm']['EP'] = (7.58 * 50 / 60) * 3600 # converting to joules https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=1&memory=512&platformType=localServer
online_tools['Green Algorithm']['LU'] = (7.58 * 30 / 60) * 3600 # https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=0.7&memory=512&platformType=localServer
online_tools['Green Algorithm']['MG'] = (7.58 * 125 / 60) * 3600 # https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0.2&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=0.25&memory=512&platformType=localServer
online_tools['Green Algorithm']['idle'] = (0 * 60 / 60) * 3600 # https://green-algorithms.org//?runTime_hour=0&runTime_min=1&appVersion=v2.2&locationContinent=Europe&locationCountry=France&locationRegion=FR&PUEradio=Yes&PUE=1&coreType=Both&numberCPUs=40&CPUmodel=Xeon%20E5-2697%20v4&usageCPUradio=Yes&usageCPU=0&numberGPUs=8&GPUmodel=NVIDIA%20Tesla%20V100&usageGPUradio=Yes&usageGPU=0&memory=512&platformType=localServer

In [60]:
online_tools

{'Wattmetre': {'EP': 58014.00000000001,
  'LU': 35114.4,
  'MG': 154180.8,
  'idle': 69217.2},
 'BMC': {'EP': 48024.0,
  'LU': 26712.000000000004,
  'MG': 134028.0,
  'idle': 57524.4},
 'Green Algorithm': {'EP': 22740.0, 'LU': 13644.0, 'MG': 56850.0, 'idle': 0.0},
 'ML CO2 Impact': {'EP': 290000, 'LU': 174000, 'MG': 725000, 'idle': 0}}

In [61]:
# GPU
online_tools['ML CO2 Impact'] = {}
online_tools['ML CO2 Impact']['EP'] = (300*1*8 + 145*0*40 ) * 68 
online_tools['ML CO2 Impact']['LU'] = (300*0.7*8 + 145*0*40 ) * 204 
online_tools['ML CO2 Impact']['MG'] = (300*0.25*8 + 145*0.2*40 ) * 157
online_tools['ML CO2 Impact']['idle'] = 0

# CPU
online_tools['ML CO2 Impact'] = {}
online_tools['ML CO2 Impact']['EP'] = (135*1*2 ) * 50 
online_tools['ML CO2 Impact']['LU'] = (135*1*2) * 30 
online_tools['ML CO2 Impact']['MG'] = (135*1*2 ) * 125
online_tools['ML CO2 Impact']['idle'] = 0

In [62]:
online_tools

{'Wattmetre': {'EP': 58014.00000000001,
  'LU': 35114.4,
  'MG': 154180.8,
  'idle': 69217.2},
 'BMC': {'EP': 48024.0,
  'LU': 26712.000000000004,
  'MG': 134028.0,
  'idle': 57524.4},
 'Green Algorithm': {'EP': 22740.0, 'LU': 13644.0, 'MG': 56850.0, 'idle': 0.0},
 'ML CO2 Impact': {'EP': 13500, 'LU': 8100, 'MG': 33750, 'idle': 0}}

In [63]:
import pandas as pd
online_tool_df = pd.DataFrame(online_tools)
online_tool_df

,Wattmetre,BMC,Green Algorithm,ML CO2 Impact
EP,58014.0,48024.0,22740.0,13500
LU,35114.4,26712.0,13644.0,8100
MG,154180.8,134028.0,56850.0,33750
idle,69217.2,57524.4,0.0,0


In [64]:
online_tool_df['Offset_GA'] = abs(online_tool_df['Wattmetre'] - online_tool_df['Green Algorithm'])
online_tool_df['Offset_MCI'] = abs(online_tool_df['Wattmetre'] - online_tool_df['ML CO2 Impact'])
online_tool_df

,Wattmetre,BMC,Green Algorithm,ML CO2 Impact,Offset_GA,Offset_MCI
EP,58014.0,48024.0,22740.0,13500,35274.0,44514.0
LU,35114.4,26712.0,13644.0,8100,21470.4,27014.4
MG,154180.8,134028.0,56850.0,33750,97330.8,120430.8
idle,69217.2,57524.4,0.0,0,69217.2,69217.2


In [74]:
new_cols = ['Wattmetre', 'BMC', 'Tool', 'Offset \n(Wattmetre - Tool)', 'tool_name', 'gpu_0_appli','gpu_0_appli_class']

In [75]:
online_tool_df

,Wattmetre,BMC,Green Algorithm,ML CO2 Impact,Offset_GA,Offset_MCI
EP,58014.0,48024.0,22740.0,13500,35274.0,44514.0
LU,35114.4,26712.0,13644.0,8100,21470.4,27014.4
MG,154180.8,134028.0,56850.0,33750,97330.8,120430.8
idle,69217.2,57524.4,0.0,0,69217.2,69217.2


## Format matching the rest of the experiments
To copy to the Energy notebook.

In [76]:
# GPU
to_concat = {
    'Wattmetre':[175665.6, 405903.6, 168228.0, 175665.6, 405903.6, 168228.0], 
    'BMC':[152326.8, 366130.8, 146700.0, 152326.8, 366130.8, 146700.0], 
    'Tool':[176174.4, 381643.2, 134329.2, 163200.0, 342720.0, 276320.0], 
    'Offset \n|Wattmetre - Tool|':[508.8, 24260.4, 33898.8, 12465.6, 63183.6, 108092.0], 
    'tool_name':['Green Algorithm', 'Green Algorithm', 'Green Algorithm', 'ML CO2 Impact', 'ML CO2 Impact', 'ML CO2 Impact'], 
    'gpu_0_appli':['ep', 'lu', 'mg', 'ep', 'lu', 'mg'],
    'gpu_0_appli_class':['E', 'D', 'D', 'E', 'D', 'D'],
    'tool_on_one_process':[False,False,False,False,False,False]
    }
df_to_concat = pd.DataFrame(to_concat)
df_to_concat

,Wattmetre,BMC,Tool,Offset \n|Wattmetre - Tool|,tool_name,gpu_0_appli,gpu_0_appli_class,tool_on_one_process
0,175665.6,152326.8,176174.4,508.8,Green Algorithm,ep,E,False
1,405903.6,366130.8,381643.2,24260.4,Green Algorithm,lu,D,False
2,168228.0,146700.0,134329.2,33898.8,Green Algorithm,mg,D,False
3,175665.6,152326.8,163200.0,12465.6,ML CO2 Impact,ep,E,False
4,405903.6,366130.8,342720.0,63183.6,ML CO2 Impact,lu,D,False
5,168228.0,146700.0,276320.0,108092.0,ML CO2 Impact,mg,D,False


In [78]:
online_tool_df

,Wattmetre,BMC,Green Algorithm,ML CO2 Impact,Offset_GA,Offset_MCI
EP,58014.0,48024.0,22740.0,13500,35274.0,44514.0
LU,35114.4,26712.0,13644.0,8100,21470.4,27014.4
MG,154180.8,134028.0,56850.0,33750,97330.8,120430.8
idle,69217.2,57524.4,0.0,0,69217.2,69217.2


In [79]:
# CPU
to_concat = {
    'Wattmetre':[58014.0, 35114.4, 154180.8, 58014.0, 35114.4, 154180.8], 
    'BMC':[48024.0, 26712.0, 134028.0, 48024.0, 26712.0, 134028.0], 
    'Tool':[22740.0, 13644.0, 56850.0, 13500, 8100, 33750], 
    'Offset \n|Wattmetre - Tool|':[35274, 21470.4, 97330.8, 44514.0, 27014.4, 120430.8], 
    'tool_name':['Green Algorithm', 'Green Algorithm', 'Green Algorithm', 'ML CO2 Impact', 'ML CO2 Impact', 'ML CO2 Impact'], 
    'gpu_0_appli':['ep', 'lu', 'mg', 'ep', 'lu', 'mg'],
    'gpu_0_appli_class':['D', 'C', 'D', 'D', 'C', 'D'],
    'tool_on_one_process':[False,False,False,False,False,False]
    }
df_to_concat = pd.DataFrame(to_concat)
df_to_concat

,Wattmetre,BMC,Tool,tool_name,gpu_0_appli,gpu_0_appli_class,tool_on_one_process
0,58014.0,48024.0,22740.0,Green Algorithm,ep,D,False
1,35114.4,26712.0,13644.0,Green Algorithm,lu,C,False
2,154180.8,134028.0,56850.0,Green Algorithm,mg,D,False
3,58014.0,48024.0,13500.0,ML CO2 Impact,ep,D,False
4,35114.4,26712.0,8100.0,ML CO2 Impact,lu,C,False
5,154180.8,134028.0,33750.0,ML CO2 Impact,mg,D,False


In [42]:
new_cols = ['Wattmetre', 'BMC', 'Tool']#, 'Offset \n|Wattmetre - Tool|']
group_by_cols = ['tool_name']
df_to_concat.groupby(by=group_by_cols).mean()[new_cols]

,Wattmetre,BMC,Tool
tool_name,,,
Green Algorithm,82436.4,69588.0,35178.000000
ML CO2 Impact,82436.4,69588.0,396333.333333
